## Modeling Pipeline :
* 1.use item similarity to overcome cold start issue
* 2.pipeline: train model, save model, import new user data, load model for recommendation, output recommendation, 

In [2]:
import boto3
import pickle
import h5py
import tables
import numpy as np
import pandas as pd
import sklearn
import caserec
import json

In [3]:
import turicreate as tc

In [3]:
raw_data=pd.read_csv('training_data_3column_750.csv')

In [4]:
raw_data_user_count=raw_data.groupby('game_id')['user_id'].count().reset_index()

In [5]:
training_data_2k=raw_data[raw_data.game_id.isin(raw_data_user_count[raw_data_user_count.user_id>2000].game_id)]

In [6]:
training_data_2k.shape

(33618784, 4)

In [7]:
training_data_2k.game_id.nunique()

4237

In [8]:
training_data_2k.to_csv('training_data_3column_2k_new.csv')

KeyboardInterrupt: 

In [4]:
actions = tc.SFrame.read_csv('training_data_3column_2k_new.csv')
items = tc.SFrame.read_csv('game_info_750.csv')
df_items=pd.read_csv('game_info_750.csv')

Finished parsing file /Users/shujdong/w210_game_recommendation_local/modeling/training_data_3column_2k_new.csv

Parsing completed. Parsed 100 lines in 1.57016 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,int,int,str,float]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Read 1641052 lines. Lines per second: 986027

Finished parsing file /Users/shujdong/w210_game_recommendation_local/modeling/training_data_3column_2k_new.csv

...

Finished parsing file /Users/shujdong/w210_game_recommendation_local/modeling/game_info_750.csv

Parsing completed. Parsed 100 lines in 0.045713 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,int,str,list,float,int,int,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /Users/shujdong/w210_game_recommendation_local/modeling/game_info_750.csv

Parsing completed. Parsed 9595 lines in 0.051344 secs.

In [13]:
actions=actions[['game_id','user_id','rating']]
actions

game_id,user_id,rating
1,Lil Red Head,7.0
1,hsjx945,8.0
1,RuffRyder,0.0
1,SkywalterDBZ,7.0
1,Olmestig,8.0
1,grassquit,9.0
1,erzengel,0.0
1,Feeble,0.0
1,thisisadamj,0.0
1,Tiberiusfox,9.0


In [19]:


model = tc.recommender.item_similarity_recommender.create(actions,'user_id','game_id',target='rating',similarity_type='pearson')



Preparing data set.

Data has 11667592 observations with 406977 users and 1372 items.

Data prepared in: 24.8221s

Training model from provided data.

Gathering per-item and per-user statistics.

...

| 1m 7s                               | 100              | 1372            |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 68.6022s

In [20]:
similar_items = model.get_similar_items([1,3],k=5)
similar_items

game_id,similar,score,rank
1,4098,0.24386858940124512,1
1,9216,0.21843940019607544,2
1,475,0.2178107500076294,3
1,19777,0.21709173917770386,4
1,24181,0.2169148325920105,5
3,503,0.2943037152290344,1
3,12,0.2758612036705017,2
3,42,0.2736928462982178,3
3,118,0.23857837915420532,4
3,93,0.23739522695541382,5


In [22]:
#save model

model.save('game_rec_model_2k_pearson')

In [5]:
#load model
rec_model=tc.load_model('game_rec_model_2k_pearson')

In [24]:
rec_model

Class                            : ItemSimilarityRecommender

Schema
------
User ID                          : user_id
Item ID                          : game_id
Target                           : rating
Additional observation features  : 0
User side features               : []
Item side features               : []

Statistics
----------
Number of observations           : 11667592
Number of users                  : 406977
Number of items                  : 1372

Training summary
----------------
Training time                    : 68.6076

Model Parameters
----------------
Model class                      : ItemSimilarityRecommender
threshold                        : 0.001
similarity_type                  : pearson
training_method                  : auto

Other Settings
--------------
degree_approximation_threshold   : 4096
max_data_passes                  : 4096
max_item_neighborhood_size       : 64
nearest_neighbors_interaction_proportion_threshold : 0.05
seed_item_set_size           

In [25]:
#try with random user input data
new_obs_data = tc.SFrame({ 'user_id' : ['survey', 'survey', 'survey', 'survey', 'survey'],
                                'game_id' : ['4098', '9216', '475', '148228', '68448']})

recommend_games = rec_model.recommend_from_interactions([4098,9216,475])
recommend_games

game_id,score,rank
2181,5.480710946162038,1
592,5.3227318136439346,2
6887,5.25355799036145,3
39856,5.005193770808811,4
463,4.997637687223129,5
1115,4.859914327949887,6
6901,4.8412428435517985,7
15985,4.801910657206865,8
30618,4.6007139951179745,9
3076,4.546229831055017,10


game_id,score,rank
2181,5.480710946162038,1
592,5.3227318136439346,2
6887,5.25355799036145,3
39856,5.005193770808811,4
463,4.997637687223129,5
1115,4.859914327949887,6
6901,4.8412428435517985,7
15985,4.801910657206865,8
30618,4.6007139951179745,9
3076,4.546229831055017,10


In [26]:
new_obs_data_1 = [362, 3, 9440]

#recommend_games_1 = rec_model.recommend_from_interactions([362,3,9440],k=50)
recommend_games_1 = rec_model.recommend_from_interactions(new_obs_data_1,k=50)

recommend_games_1

game_id,score,rank
2181,5.480710946162038,1
592,5.3227318136439346,2
6887,5.25355799036145,3
39856,5.005193770808811,4
463,4.997637687223129,5
1115,4.859914327949887,6
6901,4.8412428435517985,7
15985,4.801910657206865,8
30618,4.6007139951179745,9
31260,4.3773630912140895,10


In [6]:
my_list_of_items=[130802, 8577, 362, 30549, 31]
similar_items = rec_model.get_similar_items(my_list_of_items,k=3)

In [7]:
print(similar_items)

+---------+---------+---------------------+------+
| game_id | similar |        score        | rank |
+---------+---------+---------------------+------+
|  30549  |   822   | 0.38645923137664795 |  1   |
|  30549  |   9209  |  0.3811255097389221 |  2   |
|  30549  |  36218  | 0.37558168172836304 |  3   |
|    31   |   700   | 0.12949872016906738 |  1   |
|    31   |   1149  | 0.12810808420181274 |  2   |
|    31   |   1762  | 0.11922997236251831 |  3   |
+---------+---------+---------------------+------+
[6 rows x 4 columns]



In [8]:
recommend_games_n = rec_model.recommend_from_interactions(my_list_of_items,k=20)
recommend_games_n

game_id,score,rank
2181,5.480710946162038,1
592,5.3227318136439346,2
6887,5.25355799036145,3
463,4.907110152738266,4
39856,4.867760146541233,5
1115,4.859914327949887,6
6901,4.8412428435517985,7
15985,4.801910657206865,8
30618,4.6007139951179745,9
3076,4.387786105945917,10


In [9]:
recommend_games_m = rec_model.recommend_from_interactions(my_list_of_items,k=20)
recommend_games_m

game_id,score,rank
2181,5.480710946162038,1
592,5.3227318136439346,2
6887,5.25355799036145,3
463,4.907110152738266,4
39856,4.867760146541233,5
1115,4.859914327949887,6
6901,4.8412428435517985,7
15985,4.801910657206865,8
30618,4.6007139951179745,9
3076,4.387786105945917,10


In [ ]:
##backend flask import new user data and return 
from flask import Flask
from flask import request
app = Flask(__name__)

import turicreate as tc

import json

rec_model=tc.load_model('game_rec_model')

@app.route("/recommend", methods=['POST'])

In [16]:
#function to import json of user's answers, run model prediction, 
def recommend():
#import new user survey answer
#new_obs_data = json.loads("../data/bgg_survey_result.json")

    json_file_path = "../data/questionnaire_result.json"

    with open(json_file_path, 'r') as j:
         new_obs_data = json.loads(j.read())

    new_obs_data_new=[int(i) for i in list(new_obs_data["games"].keys())]
    print(new_obs_data)
    filter_condt=tc.SFrame({"age_min": [new_obs_data["age"]["min"]],
                        "age_max": [new_obs_data["age"]["max"]],
                        "num_players_min": [new_obs_data["num_players"]["min"]],
                        "num_players_max": [new_obs_data["num_players"]["max"]],
                        "play_time_min": [new_obs_data["play_time"]["min"]],
                        "play_time_max": [new_obs_data["play_time"]["max"]]})
    recommend_items = rec_model.recommend_from_interactions(new_obs_data_new,k=50)
    
    #select 50 recommended game info 
    df_rec_game_info=df_items.loc[df_items['game_id'].isin(recommend_items['game_id'])]
    #filter out game based on user answers 
    df_items_filter=df_rec_game_info[(df_rec_game_info['age_min'] > filter_condt['age_min'])&
                                #(df_rec_game_info['min_ppl'] < filter_condt['num_players_min']) &
                                #(df_rec_game_info['max_ppl'] < filter_condt['num_players_max']) ]
                                (df_rec_game_info['avg_time']> filter_condt['play_time_min'])&(df_rec_game_info['avg_time']< filter_condt['play_time_max'])]
    #conver recommend_items to dataframe

    df_recommend_items=recommend_items.to_dataframe()
    #output top 5 games
    output=df_recommend_items.loc[df_recommend_items['game_id'].isin(df_items_filter.game_id)].sort_values('score',ascending=False).head(5)
    output_name=df_items_filter[df_items_filter['game_id'].isin(output['game_id'])][['game_id','game_title']]
    output=pd.merge(output,output_name, on='game_id')

    print(df_recommend_items[df_recommend_items['game_id'].isin(df_items_filter.game_id)].sort_values('score',ascending=False))
    output['level']= output['score'].where(~(output['score']>=5),"Extremely Similar")
    output['level']= output['level'].where(~((output['score']>=4.5)&(output['score']<5)),"Very Similar")
    output['level']= output['level'].where(~(output['score']<4.5),"Similar")
    json_output=json.dumps({"game_id": list(output["game_id"]), "game_name": list(output["game_title"]),"level": list(output["level"])})
    return json_output

In [17]:
df_items[df_items['game_id']==178900]


,Unnamed: 0,game_id,game_title,category,avg_time,min_ppl,max_ppl,age_min
6856,77039,178900,Codenames,"['Card Game', 'Deduction', 'Party Game', 'Spie...",15.0,2.0,8.0,14.0


In [18]:
#score=cosin similarity
recommend()


{'survey': 'questionnaire', 'age': {'question': 'age', 'min': 10, 'max': 16}, 'num_players': {'question': 'num_players', 'min': 5, 'max': 100}, 'play_time': {'question': 'play_time', 'min': 5, 'max': 100000}, 'games': {'230802': 'Azul', '9209': 'Ticket to Ride', '36218': 'Dominion', '30549': 'Pandemic', '31260': 'Agricola'}}
    game_id     score  rank
0      2181  5.480711     1
3      1115  4.859914     4
5     15985  4.801911     6
6       463  4.799032     7
9       177  4.374275    10
15     2842  4.055818    16
16       12  4.037333    17
19      712  3.980188    20
20    33160  3.967403    21
22      555  3.948287    23
23      491  3.941372    24
27     6472  3.902900    28
32    24181  3.883656    33
34     3072  3.862165    35
35     1353  3.850919    36
37       94  3.833650    38
39      760  3.817013    40
40    18602  3.813954    41
41       60  3.798081    42
42     2122  3.794293    43
45       93  3.780895    46
48     2163  3.765226    49


'{"game_id": [2181, 1115, 15985, 463, 177], "game_name": ["Bridge", "Poker", "Blood Bowl: Living Rulebook", "Magic: The Gathering", "Advanced Civilization"], "level": ["Extremely Similar", "Very Similar", "Very Similar", "Very Similar", "Similar"]}'